In [1]:
import sys
import os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

os.environ['JAX_PLATFORM_NAME'] = "cpu"
os.environ['JAX_PLATFORMS'] = "cpu"

In [2]:
import jax.numpy as jnp
import jax

import numpy as onp

import matplotlib.pyplot as plt
import random

from simulation.simulate_full import run_entire_simulation

from analyzers import defaultvalues as dv, database, loss as loss_anaylzer, gradients as grad_analyzer

database.set_filename("../data/grad_analyzer/nmolecules_scan.npz")

/home/florian/anaconda3/envs/jaxidp/lib/python3.11/site-packages/google/colab/data_table.py:30: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  from IPython.utils import traitlets as _traitlets


In [4]:
N_GRADIENTS = 4
nmolecules_values = onp.array([3, 4, 5, 6, 7, 8])
print(nmolecules_values)

[ 27  64 125 216 343 512]


In [4]:
def print_info(grads):
    amean, astd = grad_analyzer.analyze_gradients_absolute(grads)
    mmean, mstd = grad_analyzer.analyze_gradients_magnitudal(grads)

    print(f"Absolute mean: {amean}, Absolute std: {astd}")
    print(f"Magnitudal mean: {mmean}, Magnitudal std: {mstd}")

In [ ]:
for nmolecules in nmolecules_values:
    existing_keys = database.get_existing_keys()
    if nmolecules in existing_keys:
        print(f"Skipping {nmolecules}, was already computed")
        continue

    grads = []
    print(f"==== for {nmolecules}**3={nmolecules**3} molecules ====")
    for i in range(1, N_GRADIENTS + 1):
        key = random.randrange(0, 20000)
        def simulation_wrapper(LJ_SIGMA_OO: float) -> float:
            prediction = run_entire_simulation(
                LJ_SIGMA_OO, 
                dv.N_STEPS, 
                nmolecules, 
                dv.N_SNAPSHOTS, 
                dv.N_Q, 
                key)
            reference = run_entire_simulation(
                dv.LJ_SIGMA_OO, 
                dv.N_STEPS, 
                nmolecules, 
                dv.N_SNAPSHOTS, 
                dv.N_Q, 
                key)
            return loss_anaylzer.L1_loss(prediction, reference)
        grad_fn = jax.grad(simulation_wrapper)
        grad = grad_fn(3.1)
        print(grad)
        grads.append(grad)

    print_info(grads)
    database.save_intermediate_result(nmolecules, grads)

In [ ]:
keys, values = database.load_result()
for key, grads in zip(keys, values):
    print(f"==== for {nmolecules}**3={nmolecules**3} molecules ====")
    print_info(grads)